In [2]:
!pip install nlpia

     |████████████████████████████████| 32.0MB 148kB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 716kB 42.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149825 sha256=211b5008d5d83a8a9c7a0f0e8bfd54510bb598b8decd105dc686cf0f40465b59
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
  Created wheel for pypandoc: filename=pypandoc-1.5-cp37-none-any.whl size=17036 sha256=b83182dd13d7b55aa305941e4c7e63b834f39919ab5515c7dfb8eabe14a78d85
  Stored in directory: /root/.cache/pip/wheels/bb/7d/d6/2f9af55e800d37e42e546106bcbd36a86e24e725e303d17e04
Successfully built python-Levenshtein pypandoc


In [3]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D

# read data
import glob
import os
from random import shuffle

from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
from nlpia.loaders import get_data

/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/usr/local/lib/python3.7/dist-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/usr/local/lib/python3.7/dist-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/usr/local/lib/python3.7/dist-packages/nlpia

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [7]:
filepath = '/content/drive/MyDrive/sentiments/aclImdb/train'

In [9]:
# load data
def pre_process_data(filepath):

    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
            
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))

    shuffle(dataset)

    return dataset

dataset = pre_process_data(filepath)

In [ ]:
# Vectorizer and tokenizer
word_vectors = get_data('w2v', limit=200000)
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
          try:
            sample_vecs.append(word_vectors[token])
          except KeyError:
            pass # No matching token in the Google w2v vocab

        vectorized_data.append(sample_vecs)

    return vectorized_data

def collect_expected(dataset):
    """ Peel off the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

100%|██████████| 402111/402111 [00:15<00:00, 25896.96it/s]

2021-04-19 10:27:50,459 WARNING:  nlpia.loaders:528:normalize_ext_rename normalize_ext.filepath=/usr/local/lib/python3.7/dist-packages/nlpia/bigdata/googlenews-vectors-negative300.bin.gz


In [ ]:
# train/test split
split_point = int(len(vectorized_data)*.8)
x_train = vectorized_data[:split_point_]
y_train_ = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

# CNN params
maxlen = 400
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [ ]:
def pad_trunc(data, maxlen):
    """
    For a given dataset pad with zero vectors or truncate to maxlen
    """
    new_data = []
    
    # Create a vector of 0s the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
    	zero_vector.append(0.0)
    
    for sample in data:
        if len(sample) > maxlen:
        	temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            # Append the appropriate number 0 vectors to the list
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
            	temp.append(zero_vector)
        else:
        	temp = sample
        new_data.append(temp)
    return new_data

#  pass train and test data into the padder/truncator
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)


	1. For each input example, you applied a filter (weights and activation function).
    2. Convolved across the length of the input, which would output a 1D vector
    slightly smaller than the original input (1 x 398 which is input with the filter
    starting left-aligned and finishing right-aligned) for each filter.
    3. For each filter output (there are 250 of them, remember), you took the single
    maximum value from each 1D vector.
    4. At this point you have a single vector (per input example) that is 1 x 250 (the
    number of filters).


In [ ]:
# CNN model
model = Sequential()
model.add(Conv1D(
                filters,
                kernel_size,
                padding='valid',
                activation='relu',
                strides=1,
                input_shape=(maxlen, embedding_dims)))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [ ]:
# save model
model_structure = model.to_json()
with open("cnn_model.json", "w") as json_file:
	json_file.write(model_structure)
model.save_weights("cnn_weights.h5")

In [ ]:
# using model
from keras.models import model_from_json
with open("cnn_model.json", "r") as json_file:
    json_string = json_file.read()
    
model = model_from_json(json_string)
model.load_weights('cnn_weights.h5')

vec_list = tokenize_and_vectorize([(1, sample_1)])
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
model.predict(test_vec)  # model.predict_classes(test_vec)